<a href="https://colab.research.google.com/github/ANGELYNEDEVSENIOR/Actividad2/blob/main/MiPrimerPhyton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Libro:
    def __init__(self, titulo, autor, isbn):
        self.titulo = titulo
        self.autor = autor
        self.isbn = isbn
        self.disponible = True

    def __str__(self):
        return f"{self.titulo} por {self.autor} (ISBN: {self.isbn})"


class Usuario:
    def __init__(self, nombre, id_usuario):
        self.nombre = nombre
        self.id_usuario = id_usuario
        self.libros_prestados = []

    def __str__(self):
        return f"{self.nombre} (ID: {self.id_usuario})"


# Nodo del árbol binario: cada nodo contiene un libro y referencias a subnodos izquierdo y derecho
class NodoLibro:
    def __init__(self, libro):
        self.libro = libro       # Guarda el objeto libro en este nodo
        self.izquierda = None    # Subárbol izquierdo (libros con título "menor")
        self.derecha = None      # Subárbol derecho (libros con título "mayor")

# Clase que implementa el Árbol Binario de Búsqueda de libros, ordenados por título
class ArbolLibros:
    def __init__(self):
        self.raiz = None  # Nodo raíz del árbol, Inicialmente el árbol está vacío.


    # Inserta un nuevo libro en el árbol
    def insertar(self, libro):
      # Función auxiliar recursiva para insertar en el lugar correcto
        def _insertar(nodo, libro):  # Si el nodo actual es None, crea un nuevo nodo con el libro
            if nodo is None:
                return NodoLibro(libro)
            if libro.titulo < nodo.libro.titulo:
                nodo.izquierda = _insertar(nodo.izquierda, libro) # Va a la izquierda si el título es menor
            else:
                nodo.derecha = _insertar(nodo.derecha, libro)  # Va a la derecha si es igual o mayor
            return nodo
        self.raiz = _insertar(self.raiz, libro)  # Comienza desde la raíz

      # Busca un libro por título
    def buscar(self, titulo):
      # Función auxiliar recursiva de búsqueda
        def _buscar(nodo, titulo):
            if nodo is None:  # No encontrado
                return None
            if nodo.libro.titulo == titulo:
                return nodo.libro  # Libro encontrado
            elif titulo < nodo.libro.titulo:
                return _buscar(nodo.izquierda, titulo)  # Va a la izquierda si el título es menor
            else:
                return _buscar(nodo.derecha, titulo)  # Va a la derecha si es mayor
        return _buscar(self.raiz, titulo)

   # Imprime todos los libros en orden alfabético por título (recorrido inorden)
    def imprimir_en_orden(self):
        def _inorden(nodo):
            if nodo:
                _inorden(nodo.izquierda)  # Recorre el subárbol izquierdo
                print(nodo.libro)         # Luego imprime el libro actual
                _inorden(nodo.derecha)    # Finalmente recorre la derecha
        _inorden(self.raiz)


class Biblioteca:
    def __init__(self):
        self.libros = {}  # Diccionario: {isbn: libro}
        self.usuarios = {}  # Diccionario: {id_usuario: usuario}
        self.cola_reservas = []
        self.historial = []
        self.arbol_titulos = ArbolLibros()  # Árbol por título

    def agregar_libro(self, libro):
        self.libros[libro.isbn] = libro
        self.arbol_titulos.insertar(libro)

    def agregar_usuario(self, usuario):
        self.usuarios[usuario.id_usuario] = usuario

    def prestar_libro(self, isbn, id_usuario):
        if isbn in self.libros and id_usuario in self.usuarios:
            libro = self.libros[isbn]
            usuario = self.usuarios[id_usuario]
            if libro.disponible:
                libro.disponible = False
                usuario.libros_prestados.append(libro)
                self.historial.append(("prestar", libro, usuario))
                print(f"Libro '{libro.titulo}' prestado a {usuario.nombre}")
            else:
                print("El libro no está disponible.")
        else:
            print("Libro o usuario no encontrado.")

    def devolver_libro(self, isbn, id_usuario):
        if isbn in self.libros and id_usuario in self.usuarios:
            libro = self.libros[isbn]
            usuario = self.usuarios[id_usuario]
            if libro in usuario.libros_prestados:
                libro.disponible = True
                usuario.libros_prestados.remove(libro)
                self.historial.append(("devolver", libro, usuario))
                print(f"Libro '{libro.titulo}' devuelto por {usuario.nombre}")
            else:
                print("El libro no está prestado a este usuario.")
        else:
            print("Libro o usuario no encontrado.")

    def reservar_libro(self, isbn, id_usuario):
        if isbn in self.libros and id_usuario in self.usuarios:
            libro = self.libros[isbn]
            usuario = self.usuarios[id_usuario]
            self.cola_reservas.append((libro, usuario))
            print(f"Libro '{libro.titulo}' reservado por {usuario.nombre}")
        else:
            print("Libro o usuario no encontrado.")

    def atender_reserva(self):
        if self.cola_reservas:
            libro, usuario = self.cola_reservas.pop(0)
            self.prestar_libro(libro.isbn, usuario.id_usuario)
        else:
            print("No hay reservas pendientes.")

    def deshacer(self):
        if self.historial:
            operacion, libro, usuario = self.historial.pop()
            if operacion == "prestar":
                libro.disponible = True
                usuario.libros_prestados.remove(libro)
                print(f"Operación de préstamo de '{libro.titulo}' deshecha.")
            elif operacion == "devolver":
                libro.disponible = False
                usuario.libros_prestados.append(libro)
                print(f"Operación de devolución de '{libro.titulo}' deshecha.")
        else:
            print("No hay operaciones para deshacer.")


def validar_isbn():
    while True:
        isbn = input("ISBN del libro (4 dígitos): ")
        if isbn.isdigit() and len(isbn) == 4:
            return isbn
        else:
            print("ISBN inválido. Debe ser un número de 4 dígitos. Inténtelo de nuevo.")

def validar_id_usuario():
    while True:
        id_usuario = input("ID del usuario: ")
        if id_usuario.isdigit():
            return id_usuario
        else:
            print("ID inválido. Debe ser un número. Inténtelo de nuevo.")

def mostrar_menu():
    biblioteca = Biblioteca()
    while True:
        print("\n *** Menú de la Biblioteca ***")
        print("1. Agregar usuario")
        print("2. Agregar libro")
        print("3. Prestar libro")
        print("4. Devolver libro")
        print("5. Reservar libro")
        print("6. Mostrar libros prestados por usuario")
        print("7. Buscar libro por título (ABB)")
        print("8. Mostrar libros en orden alfabético (ABB)")
        print("9. Salir")

        opcion = input("Ingrese la opción deseada: ")
        try:
            opcion = int(opcion)
        except ValueError:
            print("Opción inválida. Debe ser un número.")
            continue

        if opcion == 1:
            nombre = input("Nombre del usuario: ")
            id_usuario = validar_id_usuario()
            usuario = Usuario(nombre, id_usuario)
            biblioteca.agregar_usuario(usuario)
            print("Usuario agregado.")

        elif opcion == 2:
            titulo = input("Título del libro: ")
            autor = input("Autor del libro: ")
            isbn = validar_isbn()
            libro = Libro(titulo, autor, isbn)
            biblioteca.agregar_libro(libro)
            print("Libro agregado.")

        elif opcion == 3:
            isbn = validar_isbn()
            id_usuario = validar_id_usuario()
            biblioteca.prestar_libro(isbn, id_usuario)

        elif opcion == 4:
            isbn = validar_isbn()
            id_usuario = validar_id_usuario()
            biblioteca.devolver_libro(isbn, id_usuario)

        elif opcion == 5:
            isbn = validar_isbn()
            id_usuario = validar_id_usuario()
            biblioteca.reservar_libro(isbn, id_usuario)

        elif opcion == 6:
            id_usuario = validar_id_usuario()
            usuario = biblioteca.usuarios.get(id_usuario)
            if usuario:
                libros_prestados = usuario.libros_prestados
                if libros_prestados:
                    print("Libros prestados:")
                    for libro in libros_prestados:
                        print(libro)
                else:
                    print("El usuario no tiene libros prestados.")
            else:
                print("Usuario no encontrado.")

        elif opcion == 7:
            titulo = input("Título del libro a buscar: ")
            libro = biblioteca.arbol_titulos.buscar(titulo)
            if libro:
                print("Libro encontrado:")
                print(libro)
            else:
                print("No se encontró el libro.")

        elif opcion == 8:
            print("Libros en orden alfabético por título:")
            biblioteca.arbol_titulos.imprimir_en_orden()

        elif opcion == 9:
            print("¡Hasta luego!")
            break

        else:
            print("Opción inválida. Inténtelo de nuevo.")

mostrar_menu()